In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.utils import shuffle
import utility as ut

data = pd.read_csv('synthetic.csv')
# Afficher les premières lignes pour s'assurer que les données sont correctement chargées
print(data.head())

In [ ]:
df_columns = data.columns.values.tolist()
features = df_columns[0:14]
label = df_columns[14:] 

X = data[features]
y = data[label]

In [ ]:
x = data[features]
y = data[label]
y = pd.get_dummies(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42) 

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.15)

In [ ]:
def relu(x):
    return np.maximum(0, x)

def softmax(z):
    exp_scores = np.exp(z)
    return exp_scores / np.sum(exp_scores, axis=0)


In [ ]:

class NeuralNetwork:
    def __init__(self, X_train=None, y_train=None, X_test=None, y_test=None,
                 hidden_layer_sizes=[10, 8, 6], activation='tanh', learning_rate=0.01, epoch=200):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = np.tanh
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.n_layers = len(hidden_layer_sizes)
        self.w = [np.array([])] * (self.n_layers+1)
        self.b = [np.array([])] * (self.n_layers+1)
        self.df = [None]*(self.n_layers+1)
        self.A = [np.array([])]*(self.n_layers+1)
        self.Z = [None] * (self.n_layers+1)

        l = [X_train.shape[1]] + hidden_layer_sizes + [y_train.shape[1]]

        for i in range(0, len(l)-1):
            self.w[i], self.b[i] = self.weights_initialization(l[i], l[i+1])
    

    def weights_initialization(self, n_attributs, n_classes):
        weights = np.random.uniform(low=0.0, high=1.0, size=(n_classes, n_attributs))
        bias = np.random.uniform(low=0.0, high=1.0, size=(n_classes, 1))
        return weights, bias




    def __cross_entropy(self, weights, A, Z):
        entropy = 0
        for i in len(self.weights):
            entropy += self.Z[i] * math.log(self.A[i], 2)
        return -entropy

    def __forward_pass(self, X):
        # Transposer les données d'entrée
        X = X.transpose()

        for i in range(self.n_layers):
            # Calculer la sortie de la couche cachée
            self.Z[i+1] = np.dot(self.w[i+1], self.A[i]) + self.b[i+1]
            self.A[i+1] = relu(self.Z[i+1])

        # Calculer la sortie de la couche de sortie avec softmax
        self.Z[self.n_layers] = np.dot(self.w[self.n_layers], self.A[self.n_layers-1]) + self.b[self.n_layers]
        self.A[self.n_layers] = softmax(self.Z[self.n_layers])

        return self.A[self.n_layers]


    def backward_pass(self, X, y):
        L = self.n_layers
        delta = [np.array([])] * (self.n_layers + 1)
        dw = [np.array([])] * (self.n_layers + 1)
        db = [np.array([])] * (self.n_layers + 1)

        delta[L] = self.A[L] - y
        dw[L] = np.dot(delta[L], self.A[L-1].transpose())
        db[L] = delta[L]

        for i in range(L-1, 0, -1):  # Parcours des couches cachées
            delta[i] = np.multiply(np.dot(self.w[i+1].transpose(), delta[i+1]), self.df[i])
            db[i] = delta[i]
            dw[i] = np.dot(delta[i], self.A[i-1].transpose())

        # Calcul des gradients pour la première couche cachée
        delta[0] = np.multiply(np.dot(self.w[1].transpose(), delta[1]), self.df[0])
        db[0] = delta[0]
        dw[0] = np.dot(delta[0], X.transpose())

        # Mise à jour des poids et des biais
        for i in range(self.n_layers):
            self.w[i] -= self.learning_rate * dw[i]
            self.b[i] -= self.learning_rate * db[i]
    
    def cross_entropy_loss(y_true, y_pred):

        assert y_true.shape == y_pred.shape, "Shapes of y_true and y_pred do not match"
        
        # Nombre d'échantillons
        m = y_true.shape[0]
        
        # Calcul de la perte d'entropie croisée
        loss = -np.sum(y_true * np.log(y_pred + 1e-15)) / m  # Ajout de epsilon pour éviter les divisions par zéro
        
        return loss


def epoch(self):
    indices_train = np.random.permutation(len(self.X_train))
    X_train_shuffled = self.X_train[indices_train]
    y_train_shuffled = self.y_train[indices_train]
    
    error_train = []
    for i in range(len(X_train_shuffled)):
        prediction_train = self.forward_pass_instance(X_train_shuffled[i])
        error_train.append(self.cross_entropy_loss(y_train_shuffled[i], prediction_train))
        self.backpropagation_and_update(X_train_shuffled[i], y_train_shuffled[i])
    mean_error_train = np.mean(error_train)

    error_test = []
    for i in range(self.n_layers):
        self.w[i], self.b[i] = self.__weights_initialization(l[i], l[i+1])


    return mean_error_train


import matplotlib.pyplot as plt

def afficher(epochs, nn):
    errors_train_l = []
    errors_test_l = []
    epochs_l = np.arange(1, epochs+1)
    
    for i in range(epochs):
        err_train, err_test = nn.epoch()
        errors_train_l.append(err_train)
        errors_test_l.append(err_test)

    plt.plot(epochs_l, errors_train_l, color='r', label='train') 
    plt.plot(epochs_l, errors_test_l, color='g', label='test') 
    plt.title('Evolution of error during training')
    plt.ylabel('Erreur')
    plt.xlabel('Epoch of training')
    plt.legend() 
    plt.show()



In [ ]:
test = NeuralNetwork(X_train, y_train, X_validation, y_validation, hidden_layer_sizes=[10, 8, 6], activation=np.tanh, learning_rate=0.01)

# Initialisation des poids et des biais
for i in range(len(test.w)):
    test.w[i], test.b[i] = test.weights_initialization(n_attributs=14, n_classes=4)


afficher(50,test)